In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
%matplotlib inline
from life_utils import to_numpy, autoguess_life_file, zpad, life_numpy
from life_anim import load_lif, make_gif, life_anim, show_lif, get_cmap
import IPython.display
import scipy.ndimage

In [ ]:
pat = load_lif("patterns/gun30.lif")
anim = life_anim(pat, 240, pad=15, boundary="fill")

In [ ]:
make_gif(anim, fps=10, translate=(-0.25, -0.25), label="Translating")

In [ ]:
from __future__ import print_function, division
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
## Parameters

nu_f = 0.01  # viscosity
rho_0 = 1.0 # initial density

In [ ]:
## Dimensions

t_scale = 1
nt = len(anim) * t_scale  # time steps
nx = anim[0].shape[1]  # dimensions
nz = anim[1].shape[1]

In [ ]:
c = np.array(
    [[0, 0], [1, 0], [-1, 0], [0, 1], [0, -1], [1, 1], [-1, -1], [1, -1], [-1, 1]]
)
ai = np.array([0, 2, 1, 4, 3, 6, 5, 8, 7])  # inverse indices

na = 9  # connections (velocities)
D = 2  # dimension

# weights
w0 = 4.0 / 9.0
w1 = 1.0 / 9.0
w2 = 1.0 / 36.0
w = np.array([w0, w1, w1, w1, w1, w2, w2, w2, w2])

dt = 1
dx = 1
S = dx / dt
c1 = 1.0
c2 = 3.0 / (S ** 2)
c3 = 9.0 / (2.0 * S ** 4)
c4 = -3.0 / (2.0 * S ** 2)

tau_f = nu_f * 3.0 / (S * dt) + 0.5

# allocate matrices
f = np.zeros((na, nz, nx))
f_stream = np.zeros((na, nz, nx))
f_eq = np.zeros((na, nz, nx))
Delta_f = np.zeros((na, nz, nx))
rho = np.ones((nz, nx))  # density
u = np.zeros((D, nz, nx))  # velocity
Pi = np.zeros((D, nz, nx))
u2 = np.zeros((D, nz, nx))
cu = np.zeros((D, nz, nx))
solid = np.zeros((na, nz, nx))

#solid[[1,5,7], 20:35, :] = 1.0

# set up initial density
rho[:] = rho_0

xs,zs = np.meshgrid(np.arange(nx), np.arange(nz))

for a in range(na):
    f[a] = rho * w[a]

    # vector indices
indexes = np.zeros((na, nx * nz), dtype=np.uint32)
for a in range(na):
    xArr = (np.arange(nx) - c[a][0] + nx) % nx
    zArr = (np.arange(nz) - c[a][1] + nx) % nz    
    
    xInd, zInd = np.meshgrid(xArr, zArr)
    indTotal = zInd * nx + xInd
    indexes[a] = indTotal.reshape(nx * nz)

In [ ]:

frames = []

for t in np.arange(nt + 1):
    
    frame_ix = int(t / t_scale)
    
    diff = anim[frame_ix-1] - anim[frame_ix-2]
    
    
    # periodic boundary conditions
    f[:, :, 0] = f[:, :, -2]
    f[:, :, -1] = f[:, :, 1]
    
    # solid[:, :diff.shape[0], :diff.shape[1]] = anim[frame_ix-1]
    
    # stream
    for a in range(na):
        f_new = f[a].reshape(nx * nz)[indexes[a]]
        f_bounce = f[ai[a]]
        f_stream[a] = solid[a] * f_bounce + (1 - solid[a]) * f_new.reshape(nz, nx)
    f = f_stream.copy()

    # rho and u
    rho = np.sum(f, axis=0)
    rho[:diff.shape[0], :diff.shape[1]] += diff * 0.1
    Pi = np.einsum("azx, ad->dzx", f, c)
    u[0:D] = Pi[0:D] / rho

    # equilibrium distiribtion
    u2 = u[0] ** 2 + u[1] ** 2
    for a in range(na):
        cu = c[a][0] * u[0] + c[a][1] * u[1]
        f_eq[a] = rho * w[a] * (c1 + c2 * cu + c3 * cu ** 2 + c4 * u2)

    # collision
    Delta_f = (f_eq - f) / tau_f
    f += Delta_f
    frames.append(np.array(rho))

In [ ]:
plt.imshow(frames[120])

In [ ]:
f = np.array(frames)
cm = get_cmap('viridis')
make_gif(cm(f*0.5), fps=60)